In [28]:
from __future__ import print_function
import sys
import re
import numpy as np

from numpy import dot
from numpy.linalg import norm
from pyspark import SparkContext
from operator import add
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.regression import LabeledPoint
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from time import time

Dimention=20000
smallTrainFile='SmallTrainingData.txt'
TestFile='TestingData.txt'

In [29]:
def freqArray (listOfIndices, numberofwords):
    returnVal = np.zeros (Dimention)
    for index in listOfIndices:
        returnVal[index] = returnVal[index] + 1
    returnVal = np.divide(returnVal, numberofwords)
    return returnVal


In [30]:
sc.stop()
# load training set
start_time = time()
sc = SparkContext(appName="SVM")

d_corpus = sc.textFile(smallTrainFile)
t_d_corpus = sc.textFile(TestFile)


numberOfDocs =d_corpus.count()

d_keyAndText = d_corpus.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))
regex = re.compile('[^a-zA-Z]')
d_keyAndListOfWords = d_keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

allWords = d_keyAndListOfWords.flatMap(lambda x: ((i,1) for i in x[1]))

allCounts = allWords.reduceByKey(lambda x,y: x+y)
allCounts.cache()
topWords = allCounts.top(Dimention, lambda x : x[1])

topWordsK = sc.parallelize(range(Dimention))
dictionary = topWordsK.map (lambda x : (topWords[x][0], x))

allWordsWithDocID = d_keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

allDictionaryWords = dictionary.join (allWordsWithDocID)


justDocAndPos = allDictionaryWords.map (lambda x: (x[1][1], x[1][0]))

allDictionaryWordsInEachDoc = justDocAndPos.groupByKey()

allDocsAsNumpyArrays = allDictionaryWordsInEachDoc.map(lambda x: (x[0], freqArray(x[1],len(x[1]))))

allDocsAsNumpyArrays.cache()
allDocsAsNumpyArrays.take(3)

[('AU466',
  array([0.10181311, 0.06229661, 0.0125523 , ..., 0.        , 0.        ,
         0.        ])),
 ('AU1356',
  array([0.08316993, 0.04428105, 0.02320261, ..., 0.        , 0.        ,
         0.        ])),
 ('19264299',
  array([0.076326  , 0.03751617, 0.04915912, ..., 0.        , 0.        ,
         0.        ]))]

In [31]:
# load test set
t_numberOfDocs =t_d_corpus.count()

t_d_keyAndText = t_d_corpus.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))
regex = re.compile('[^a-zA-Z]')
t_d_keyAndListOfWords = t_d_keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

t_allWords = t_d_keyAndListOfWords.flatMap(lambda x: ((i,1) for i in x[1]))

t_allCounts = t_allWords.reduceByKey(lambda x,y: x+y)
t_allCounts.cache()
t_topWords = t_allCounts.top(Dimention, lambda x : x[1])

topWordsK = sc.parallelize(range(Dimention))
t_dictionary = topWordsK.map (lambda x : (t_topWords[x][0], x))

t_allWordsWithDocID = t_d_keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

t_allDictionaryWords = t_dictionary.join (t_allWordsWithDocID)


t_justDocAndPos = t_allDictionaryWords.map (lambda x: (x[1][1], x[1][0]))

t_allDictionaryWordsInEachDoc = t_justDocAndPos.groupByKey()

t_allDocsAsNumpyArrays = t_allDictionaryWordsInEachDoc.map(lambda x: (x[0], freqArray(x[1],len(x[1]))))
t_allDocsAsNumpyArrays.cache()
print(t_allDocsAsNumpyArrays .take(3))
load_data_duration=time() - start_time

[('AU3400', array([0.08428472, 0.04616301, 0.02273404, ..., 0.        , 0.        ,
       0.        ])), ('11872834', array([0.10126582, 0.03987342, 0.0278481 , ..., 0.        , 0.        ,
       0.        ])), ('12036063', array([0.15209989, 0.07491487, 0.03291714, ..., 0.        , 0.        ,
       0.        ]))]


In [32]:
def get_label(x):
    if 'AU' in x[0]:
        return 1
    else:
        return -1
    
def get_vector(x,NR,PR):
    return (np.max([0,(NR*((1+x[0])/2)*(1-x[0]*(np.dot(w_current,x[1])))+PR*((1-x[0])/2)*(1-x[0]*(np.dot(w_current,x[1]))))]),
            np.zeros(Dimention) if (1-x[0]*(np.dot(w_current,x[1]))) < 0 else ((x[1]/2)*(PR*(1-x[0])-NR*(1+x[0]))))

In [37]:
# SVM with Weighted Loss Function
w_current=np.random.normal(0,20,Dimention)
learningRate=1
num_iteration = 300
oldCost=0
n=numberOfDocs
#lamda = 0.001

#calculate the PR and NR

label_list = allDocsAsNumpyArrays.map(lambda x: (get_label(x))).collect()
NR=label_list.count(-1)/len(label_list)
PR=label_list.count(1)/len(label_list)

# GD 
train_start_time = time()
allDocsAsNumpyArrays.cache()
for i in range(num_iteration):
   
    Y_X_RDD=allDocsAsNumpyArrays.map(lambda x: ((get_label(x), x[1])))
    
    Gradient=Y_X_RDD.map(lambda x: get_vector(x,NR,PR))
    GradientSum = Gradient.reduce(lambda x,y: ((x[0] + y[0]),(x[1] + y[1])))
    
    # calculate cost:
    cost = GradientSum[0]
    
    oldp = np.sqrt(abs(np.dot(w_current,w_current)))
    temp=w_current
       
    w_current= w_current-learningRate*(GradientSum[1])
    
    p = np.sqrt(abs(np.dot(temp,w_current)))

    #Stop if the cost is not descreasing 
    if abs(oldp-p) <= 0.00005:
        break

    if(cost<oldCost):
        learningRate=learningRate*1.05
        oldCost = cost

    if(cost>oldCost):
        learningRate=learningRate*0.5
        oldCost = cost
        

    print("Iteration No.=", i , " Cost=", cost) 
    print('w current:',w_current)
print('finish')
train_duration= time()-train_start_time  
    
   

Iteration No.= 0  Cost= 145.11275275365264
w current: [ 12.08153891  10.76553492 -10.07365552 ...  11.99531118  -1.01895661
   4.91650039]
Iteration No.= 1  Cost= 140.4883473467463
w current: [ 11.71963213  10.63806426 -10.56199631 ...  11.99507097  -1.01916376
   4.91639609]
Iteration No.= 2  Cost= 138.61147600086872
w current: [ 11.43141873  10.54919583 -11.04905313 ...  11.99481876  -1.01938128
   4.91628657]
Iteration No.= 3  Cost= 136.83950626500524
w current: [ 11.16328267  10.47226356 -11.54429628 ...  11.99455393  -1.01960966
   4.91617158]
Iteration No.= 4  Cost= 135.087746196694
w current: [ 10.98755319  10.43745949 -12.03455118 ...  11.99427586  -1.01977505
   4.91605084]
Iteration No.= 5  Cost= 133.38460556010148
w current: [ 10.83365865  10.41387003 -12.5353977  ...  11.99398389  -1.0199487
   4.91592406]
Iteration No.= 6  Cost= 131.65862171388028
w current: [ 10.70116215  10.40271556 -13.04848038 ...  11.99367732  -1.02012454
   4.91579094]
Iteration No.= 7  Cost= 129.902

Iteration No.= 59  Cost= 36.54564636820021
w current: [  9.04794043  14.32477338 -46.58631128 ...  11.98472173  -1.0451912
   4.90734177]
Iteration No.= 60  Cost= 35.99756213039871
w current: [  9.05786427  14.38888724 -46.87520631 ...  11.98472173  -1.04574696
   4.90734177]
Iteration No.= 61  Cost= 35.42764047814035
w current: [  9.11417427  14.47186436 -47.16029816 ...  11.98472173  -1.04633051
   4.90734177]
Iteration No.= 62  Cost= 34.83245015525483
w current: [  9.18106746  14.56283646 -47.45584241 ...  11.98472173  -1.04694325
   4.90734177]
Iteration No.= 63  Cost= 34.20885621263899
w current: [  9.26015154  14.66086385 -47.76195213 ...  11.98472173  -1.04758661
   4.90734177]
Iteration No.= 64  Cost= 33.64129592778247
w current: [  8.64007269  14.41049373 -48.28712792 ...  11.98472173  -1.04826215
   4.90734177]
Iteration No.= 65  Cost= 33.06263638948618
w current: [  9.09261669  14.68965014 -48.48507541 ...  11.98472173  -1.04897146
   4.90734177]
Iteration No.= 66  Cost= 32.

Iteration No.= 118  Cost= 15.161219423719546
w current: [  4.96793203  15.73206266 -58.11984274 ...  11.98472173  -1.08558238
   4.90734177]
Iteration No.= 119  Cost= 15.017874911493433
w current: [  4.53168467  15.56254253 -58.26226929 ...  11.98472173  -1.08619039
   4.90734177]
Iteration No.= 120  Cost= 14.844623875413113
w current: [  4.98851075  15.84530498 -58.12851835 ...  11.98472173  -1.0868288
   4.90734177]
Iteration No.= 121  Cost= 14.726318280519632
w current: [  4.48556937  15.6441632  -58.2906186  ...  11.98472173  -1.08749913
   4.90734177]
Iteration No.= 122  Cost= 14.505370866119481
w current: [  4.57867689  15.78174839 -58.24204119 ...  11.98472173  -1.08820298
   4.90734177]
Iteration No.= 123  Cost= 14.350772858843085
w current: [  4.30626256  15.69386134 -58.32232083 ...  11.98472173  -1.08894202
   4.90734177]
Iteration No.= 124  Cost= 14.16681487667985
w current: [  4.53543633  15.90335932 -58.22061111 ...  11.98472173  -1.08971802
   4.90734177]
Iteration No.= 

Iteration No.= 177  Cost= 8.153479211370744
w current: [  3.01243864  16.05443759 -57.30175708 ...  11.98472173  -1.10297612
   4.90734177]
Iteration No.= 178  Cost= 8.255183422723265
w current: [  7.55979618  18.33986234 -55.98511706 ...  11.98472173  -1.10297612
   4.90734177]
Iteration No.= 179  Cost= 10.84397868936956
w current: [  5.00780665  17.0882533  -56.79110856 ...  11.98472173  -1.10356009
   4.90734177]
Iteration No.= 180  Cost= 8.263907573817804
w current: [  4.292539    16.72747377 -56.99858909 ...  11.98472173  -1.10356009
   4.90734177]
Iteration No.= 181  Cost= 7.792041217921279
w current: [  3.83492148  16.52279055 -57.13613737 ...  11.98472173  -1.10356009
   4.90734177]
Iteration No.= 182  Cost= 7.666071571482499
w current: [  3.80275508  16.49760226 -57.12668195 ...  11.98472173  -1.10356009
   4.90734177]
Iteration No.= 183  Cost= 7.623603630983614
w current: [  3.77258873  16.47235735 -57.1154221  ...  11.98472173  -1.10356009
   4.90734177]
Iteration No.= 184  

Iteration No.= 236  Cost= 6.549903446302179
w current: [  3.92701268  16.80314646 -56.94827771 ...  11.98472173  -1.10356009
   4.90734177]
Iteration No.= 237  Cost= 6.546227254851855
w current: [  3.96913294  16.82494189 -56.93951862 ...  11.98472173  -1.10356009
   4.90734177]
Iteration No.= 238  Cost= 6.5413056346546234
w current: [  3.92699857  16.80760711 -56.95052274 ...  11.98472173  -1.10356009
   4.90734177]
Iteration No.= 239  Cost= 6.537808834792403
w current: [  3.97343616  16.83163656 -56.94086584 ...  11.98472173  -1.10356009
   4.90734177]
Iteration No.= 240  Cost= 6.53182654731319
w current: [  3.92698302  16.81252497 -56.95299788 ...  11.98472173  -1.10356009
   4.90734177]
Iteration No.= 241  Cost= 6.528718858787552
w current: [  4.01746577  16.86031683 -56.93300788 ...  11.98472173  -1.10356009
   4.90734177]
Iteration No.= 242  Cost= 6.530884931141048
w current: [  3.96556601  16.83884027 -56.94668798 ...  11.98472173  -1.10356009
   4.90734177]
Iteration No.= 243  

In [38]:
test_start_time=time()
y_ypred = t_allDocsAsNumpyArrays.map(
        lambda x: (1 if 'AU' in x[0] else -1, 1 if np.dot(x[1], w_current) >= 0 else -1)).collect()

y = []
ypred = []
counter = 0
for i in y_ypred:
    y.append(i[0])
    ypred.append(i[1])
print('f1 score:', f1_score(y, ypred, average='binary'))
print('confusion matrix:\n', confusion_matrix(y, ypred))

test_duration=time()-test_start_time
total_duration=time() - start_time
print('load data duration(s):%.4f, train model duration(s):%.4f, test model duration(s):%.4f total duration(s): %.4f'
    %(load_data_duration,train_duration,test_duration,total_duration))


f1 score: 0.28571428571428575
confusion matrix:
 [[18022   325]
 [  260   117]]
load data duration(s):146.1279, train model duration(s):377.2593, test model duration(s):1.5694 total duration(s): 840.0239


In [35]:
NR

0.9785008715862871

In [36]:
PR

0.02149912841371296